##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 영화 추천: TFX 내 추천 모델

참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행하기"를 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/recommenders"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/recommenders.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/recommenders.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/recommenders.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

## TFX로 포팅된 TFRS 튜토리얼

이것은 TFX 파이프라인에서 TFRS를 사용하는 방법을 설명하도록 설계된 TFX에 대한 기본적인 TensorFlow Recommenders(TFRX) 튜토리얼의 포트입니다. [기본 튜토리얼](https://www.tensorflow.org/recommenders/examples/basic_retrieval)을 반영합니다.

이해를 돕자면, 실제 추천 시스템은 종종 다음과 같은 두 단계로 구성됩니다.

1. 검색 단계는 모든 가능한 후보 중에서 후보 수백 개의 초기 세트를 선택하는 역할을 합니다. 이 모델의 주요 오브젝티브는 사용자가 관심이 없는 모든 후보를 효율적으로 제거하는 것입니다. 검색 모델은 수백 개의 후보를 다루기 때문에 계산적으로 효율적이어야 합니다.
2. 순위 단계는 검색 모델의 결과를 가지고 미세 조정하여 최적의 가능한 소수의 추천 후보를 선택합니다. 이 작업은 사용자가 관심 있어 할 만한 항목 세트를 가능성 있는 최종 후보 리스트로 줄입니다.

이 튜토리얼에서는, 첫 번째 단계인 검색에 중점을 두겠습니다. 검색 모델은 종종 다음과 같은 두 가지 하위 모델로 구성됩니다.

1. 쿼리 기능을 사용해 쿼리 표현(일반적으로 고정 차원 임베딩 백터)을 계산하는 쿼리 모델.
2. 후보 기능을 사용해 후보 표현(같은 크기의 벡터)을 계산하는 후보 보델

두 모델의 결과는 그 후 함께 곱해져 점수가 더 높을수록 후보와 쿼리 간 더 잘 맞는 것을 나타내는 쿼리 후보 선호도 점수를 산출합니다.

이 튜토리얼에서는, Movielens 데이터세트를 사용해 이러한 투타워 모델을 빌드하고 훈련하겠습니다.

다음을 수행하겠습니다.

1. MovieLens 데이터세트 수집 및 검사.
2. 검색 모델 구현.
3. 모델 훈련 및 내보내기.
4. 예측하기

## 데이터세트

Movielens 데이터세트는 미네소타 대학의 [GroupLens](https://grouplens.org/datasets/movielens/) 연구 그룹의 고전적인 데이터세트입니다. 일련의 사용자들이 영화에 부여한 일련의 평점을 포함하고 있으며 추천 시스템 연구의 주역입니다.

데이터는 다음과 같은 두 가지 방식으로 다뤄질 수 있습니다.

1. 이는 어떤 영화를 사용자가 봤는지(그리고 평점을 매겼는지) 보지 않았는지 나타내는 것으로 해석될 수 있습니다. 이것은 암묵적인 피드백의 한 형태로, 사용자의 시청 항목은 어떤 영화를 보기를 선호하고 어떤 영화를 보고 싶어 하지 않는지 알려줍니다.
2. 이는 또한 사용자가 시청한 영화를 얼마나 좋아했는지 나타내는 것으로 보일 수 있습니다. 이것은 명시적인 피드백의 한 형태입니다. 사용자가 영화를 본 것을 감안하면 사용자가 평점을 매긴 것을 봄으로써 얼마나 좋아했는지 대략적으로 알 수 있습니다.

이 튜토리얼에서는, 사용자가 볼 법한 카탈로그에서 일련의 영화를 예측하는 모델인 검색 시스템에 초점을 맞춥니다. 종종 암시적인 데이터가 여기에서는 더 유용하기 때문에 Movielens를 암시적인 시스템으로 다루겠습니다. 이는 사용자가 시청한 모든 영화는 양성 예이며 시청하지 않은 모든 영화는 암시적으로 음성 예시라는 것을 의미합니다.

## 가져오기

우선 처리된 가져오기 항목을 가져옵시다.

In [ ]:
!pip install -Uq tfx
!pip install -Uq tensorflow-recommenders
!pip install -Uq tensorflow-datasets

### shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.

In [ ]:
!pip uninstall shapely -y

### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, 위의 셀을 처음 실행할 때 런타임을 다시 시작해야 합니다(런타임 &gt; 런타임 다시 시작...). 이는 Colab이 패키지를 로드하는 방식 때문입니다.

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import apache_beam as beam

from absl import logging

from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## TFDS ExampleGen 생성하기

TensorFlow Datasets(TFDS) 데이터세트를 로드하기 위해 사용하는 [사용자 정의 ExampleGen 컴포넌트](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen)를 생성합니다. 이것은 FileBasedExampleGen에서 사용자 정의 실행자를 사용합니다.

In [ ]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.Pipeline)
@beam.typehints.with_output_types(tf.train.Example)
def _TFDatasetToExample(  # pylint: disable=invalid-name
    pipeline: beam.Pipeline,
    exec_properties: Dict[str, Any],
    split_pattern: str
    ) -> beam.pvalue.PCollection:
    """Read a TensorFlow Dataset and create tf.Examples"""
    custom_config = json.loads(exec_properties['custom_config'])
    dataset_name = custom_config['dataset']
    split_name = custom_config['split']

    builder = tfds.builder(dataset_name)
    builder.download_and_prepare()

    return (pipeline
            | 'MakeExamples' >> tfds.beam.ReadFromTFDS(builder, split=split_name)
            | 'AsNumpy' >> beam.Map(tfds.as_numpy)
            | 'ToDict' >> beam.Map(dict)
            | 'ToTFExample' >> beam.Map(utils.dict_to_example)
            )

class TFDSExecutor(BaseExampleGenExecutor):
  def GetInputSourceToExamplePTransform(self) -> beam.PTransform:
    """Returns PTransform for TF Dataset to TF examples."""
    return _TFDatasetToExample

## TFX 파이프라인 컨텍스트 초기화

In [ ]:
context = InteractiveContext()

## 데이터세트 준비하기

`FileBasedExampleGen`의 사용자 정의 실행자를 사용하여 TFDS에서 데이터세트를 로드합니다. 데이터세트가 두 개이기 때문에 두 개의 `ExampleGen` 컴포넌트를 생성합니다.

In [ ]:
# Ratings data.
ratings_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-ratings', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(ratings_example_gen, enable_cache=True)

In [ ]:
# Features of all the available movies.
movies_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-movies', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(movies_example_gen, enable_cache=True)

## `inspect_examples` 유틸리티 생성하기

편의 유틸리티를 생성하여 TF.Examples의 데이터세트를 검사합니다. 평점 데이터세트는 영화 ID, 사용자 ID, 할당된 평점, 타임 스탬프, 영화 정보 및 사용자 정보의 사전을 다음과 같이 반환합니다.

In [ ]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

inspect_examples(ratings_example_gen)

영화 데이터세트는 영화 ID, 영화 제목 및 영화가 속한 장르에 대한 데이터를 포함합니다. 장르는 정수 라벨로 인코딩됩니다.

In [ ]:
inspect_examples(movies_example_gen)

## ExampleGen이 분할했습니다

Movielens 데이터를 수집하면 `ExampleGen` 컴포넌트는 데이터를 `train` 및 `eval` 분할로 분할합니다. 이것들은 실제로 `Split-train` 및 `Split-eval`로 명명됩니다. 기본적으로 split은 66%가 훈련이며, 34%가 평가입니다.

## 영화와 평점에 대한 통계 생성하기

TFX 파이프라인의 경우 데이터세트에 대한 통계를 생성해야 합니다. [StatisticsGen 컴포넌트](https://www.tensorflow.org/tfx/guide/statsgen)를 사용하여 생성합니다. 데이터세트에 대한 스키마를 생성할 때 아래 [SchemaGen 컴포넌트](https://www.tensorflow.org/tfx/guide/schemagen)에 의해 사용될 수 있습니다. 데이터를 지속적인 기준으로 검사하고 분석하는 것이 중요하기 때문에 어쨌든 이것은 좋은 방법입니다. 데이터세트가 두 개이므로 두 개의 StatisticsGen 컴포넌트를 생성합니다.

In [ ]:
movies_stats_gen = tfx.components.StatisticsGen(
    examples=movies_example_gen.outputs['examples'])
context.run(movies_stats_gen, enable_cache=True)

In [ ]:
context.show(movies_stats_gen.outputs['statistics'])

In [ ]:
ratings_stats_gen = tfx.components.StatisticsGen(
    examples=ratings_example_gen.outputs['examples'])
context.run(ratings_stats_gen, enable_cache=True)

In [ ]:
context.show(ratings_stats_gen.outputs['statistics'])

## 영화와 평점에 대한 스키마 생성하기

TFX 파이프라인의 경우 데이터세트에서 데이터 스키마를 생성해야 합니다. [SchemaGen 컴포넌트](https://www.tensorflow.org/tfx/guide/schemagen)를 사용해 생성합니다. 이것은 더 큰 규모의 데이터세트로 확장 가능하며 변수 가공을 수행하고 훈련/서비스 스큐를 방지하는 방식으로 아래의 [Transform 컴포넌트](https://www.tensorflow.org/tfx/guide/transform)에 의해 사용됩니다. 데이터세트가 두 개이므로 두 개의 SchemaGen 컴포넌트를 생성하겠습니다.

In [ ]:
movies_schema_gen = tfx.components.SchemaGen(
    statistics=movies_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(movies_schema_gen, enable_cache=True)

In [ ]:
context.show(movies_schema_gen.outputs['schema'])

In [ ]:
ratings_schema_gen = tfx.components.SchemaGen(
    statistics=ratings_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(ratings_schema_gen, enable_cache=True)

In [ ]:
context.show(ratings_schema_gen.outputs['schema'])

## Transform을 사용하는 변수 가공

TFX 파이프라인의 구조화되고 반복 가능한 설계를 위해 변수 가공을 위한 확장 가능한 접근 방식이 필요합니다. 이를 통해 많은 추천 시스템의 일부인 대규모 데이터세트를 다룰 수 있으며 훈련/서비스 스큐도 방지할 수 있습니다. [Transform 컴포넌트](https://www.tensorflow.org/tfx/guide/transform)를 사용해 수행하겠습니다.

Transform 컴포넌트는 모듈 파일을 사용해 수행하고자 하는 변수 가공을 위한 사용자 코드를 공급하므로 해당 모듈 파일을 생성하는 것이 첫 번째 단계입니다. 데이터세트가 두 개이므로 이러한 두 개의 모듈 파일과 두 개의 Transform 컴포넌트를 생성하겠습니다.

추천에 필요한 것 중 하나는 `user_id` 및 `movie_title` 필드에 대한 어휘입니다. [basic_retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval)에는 인라인 Numpy에 생성되었지만, 여기에서는 Transform을 사용하겠습니다.

참고: 아래 `%%writefile {_movies_transform_module_file}` 셀 매직은 이 노트북이 실행 중인 노트북 서버의 파일(예: Colab VM)에 셀의 콘텐츠를 생성하고 작성합니다. 노트북 외에서 이를 수행할 때 Python 파일을 생성하기만 하면 됩니다.

In [ ]:
_movies_transform_module_file = 'movies_transform_module.py'

In [ ]:
%%writefile {_movies_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  # We only want the movie title
  return {'movie_title':inputs['movie_title']}

In [ ]:
movies_transform = tfx.components.Transform(
    examples=movies_example_gen.outputs['examples'],
    schema=movies_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_movies_transform_module_file))
context.run(movies_transform, enable_cache=True)

In [ ]:
context.show(movies_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(movies_transform, channel_name='transformed_examples')

In [ ]:
_ratings_transform_module_file = 'ratings_transform_module.py'

In [ ]:
%%writefile {_ratings_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import pdb

NUM_OOV_BUCKETS = 1

def preprocessing_fn(inputs):
  # We only want the user ID and the movie title, but we also need vocabularies
  # for both of them.  The vocabularies aren't features, they're only used by
  # the lookup.
  outputs = {}
  outputs['user_id'] = tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1')
  outputs['movie_title'] = tft.sparse_tensor_to_dense_with_shape(inputs['movie_title'], [None, 1], '-1')

  tft.compute_and_apply_vocabulary(
      inputs['user_id'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='user_id_vocab')

  tft.compute_and_apply_vocabulary(
      inputs['movie_title'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='movie_title_vocab')

  return outputs

In [ ]:
ratings_transform = tfx.components.Transform(
    examples=ratings_example_gen.outputs['examples'],
    schema=ratings_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_ratings_transform_module_file))
context.run(ratings_transform, enable_cache=True)

In [ ]:
context.show(ratings_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(ratings_transform, channel_name='transformed_examples')

## TFX에서 모델 구현하기

[basic_retrieval](https://www.tensorflow.org/recommenders/examples/basic_retrieval) 튜토리얼에서 모델은 Python 런타임의 인라인으로 생성되었습니다. TFX 파이프라인에서 모델, 메트릭 및 손실은 [Trainer라 불리는 파이프라인 컴포넌트](https://www.tensorflow.org/tfx/guide/trainer)에 대한 모듈 파일에서 정의되고 훈련됩니다. 이렇게 하면 모델, 메트릭 및 손실은 자동화하고 모니터링할 수 있는 반복 가능한 프로세스의 일부분이 됩니다.

### TensorFlow 추천 모델 아키덱처

투타워 검색 모델을 빌드하겠습니다. 투타워의 개념은 사용자 특성을 사용해 사용자 반복을 계산하는 쿼리 타워와 영화 특성을 사용해 영화 표현를 계산하는 다른 항목 타워가 있다는 것을 의미합니다. 각 타워를 별도로 빌드한 다음 (아래의 `_build_user_model()` 및 `_build_movie_model()` 메서드에서) 최종 모델(`MobieLensModel` 클래스와 같이)에서 조합할 수 있습니다. `MovieLensModel`는 `tfrs.Model` 베이스 클래스의 하위 클래스로, 모델 구축을 간소화합니다. `__init__` 메서드의 컴포넌트를 설정하고, `compute_loss` 메서드를 구현하여 원시 특성을 가져와 손실 값을 반환하기만 하면 됩니다.

In [ ]:
# We're now going to create the module file for Trainer, which will include the
# code above with some modifications for TFX.

_trainer_module_file = 'trainer_module.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import Dict, List, Text

import pdb

import os
import absl
import datetime
import glob
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_recommenders as tfrs

from absl import logging
from tfx.types import artifact_utils

from tfx import v1 as tfx
from tfx_bsl.coders import example_coder
from tfx_bsl.public import tfxio

absl.logging.set_verbosity(absl.logging.INFO)

EMBEDDING_DIMENSION = 32
INPUT_FN_BATCH_SIZE = 1


def extract_str_feature(dataset, feature_name):
  np_dataset = []
  for example in dataset:
    np_example = example_coder.ExampleToNumpyDict(example.numpy())
    np_dataset.append(np_example[feature_name][0].decode())
  return tf.data.Dataset.from_tensor_slices(np_dataset)


class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, tf_transform_output, movies_uri):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model

    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'movie_title')

    loss_metrics = tfrs.metrics.FactorizedTopK(
        candidates=movies_dataset.batch(128).map(movie_model)
        )

    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=loss_metrics
        )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    try:
      user_embeddings = tf.squeeze(self.user_model(features['user_id']), axis=1)
      # And pick out the movie features and pass them into the movie model,
      # getting embeddings back.
      positive_movie_embeddings = self.movie_model(features['movie_title'])

      # The task computes the loss and the metrics.
      _task = self.task(user_embeddings, positive_movie_embeddings)
    except BaseException as err:
      logging.error('######## ERROR IN compute_loss:\n{}\n###############'.format(err))

    return _task


# This function will apply the same transform operation to training data
# and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  try:
    transformed_features = tft_layer(raw_features)
  except BaseException as err:
    logging.error('######## ERROR IN _apply_preprocessing:\n{}\n###############'.format(err))

  return transformed_features


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  try:
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size),
      tf_transform_output.transformed_metadata.schema)
  except BaseException as err:
    logging.error('######## ERROR IN _input_fn:\n{}\n###############'.format(err))

  return None


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""
  try:
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
      """Returns the output to be used in the serving signature."""
      try:
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        result = model(transformed_features)
      except BaseException as err:
        logging.error('######## ERROR IN serve_tf_examples_fn:\n{}\n###############'.format(err))
      return result
  except BaseException as err:
      logging.error('######## ERROR IN _get_serve_tf_examples_fn:\n{}\n###############'.format(err))

  return serve_tf_examples_fn


def _build_user_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to ratings
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the query tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_user_ids = tf_transform_output.vocabulary_by_name('user_id_vocab')
    users_vocab_str = [b.decode() for b in unique_user_ids]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=users_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(users_vocab_str) + 1, embedding_dimension)
         ])
  except BaseException as err:
    logging.error('######## ERROR IN _build_user_model:\n{}\n###############'.format(err))

  return model


def _build_movie_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to movies
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the candidate tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_movie_titles = tf_transform_output.vocabulary_by_name('movie_title_vocab')
    titles_vocab_str = [b.decode() for b in unique_movie_titles]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=titles_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(titles_vocab_str) + 1, embedding_dimension)
        ])
  except BaseException as err:
      logging.error('######## ERROR IN _build_movie_model:\n{}\n###############'.format(err))
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  try:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, INPUT_FN_BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                            tf_transform_output, INPUT_FN_BATCH_SIZE)

    model = MovielensModel(
        _build_user_model(tf_transform_output, EMBEDDING_DIMENSION),
        _build_movie_model(tf_transform_output, EMBEDDING_DIMENSION),
        tf_transform_output,
        fn_args.custom_config['movies']
        )

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
  except BaseException as err:
    logging.error('######## ERROR IN run_fn before fit:\n{}\n###############'.format(err))

  try:
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['epochs'],
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during fit:\n{}\n###############'.format(err))

  try:
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    movies_artifact = fn_args.custom_config['movies'].get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'eval')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")

    movies_dataset = extract_str_feature(movies, 'movie_title')

    index.index_from_dataset(
      tf.data.Dataset.zip((
          movies_dataset.batch(100),
          movies_dataset.batch(100).map(model.movie_model))
      )
    )

    # Run once so that we can get the right signatures into SavedModel
    _, titles = index(tf.constant(["42"]))
    print(f"Recommendations for user 42: {titles[0, :3]}")

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(index,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    index.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

  except BaseException as err:
      logging.error('######## ERROR IN run_fn during export:\n{}\n###############'.format(err))

## 모델 훈련하기

모델 정의 후, [Trainer 컴포넌트](https://www.tensorflow.org/tfx/guide/trainer)를 실행하여 모델 훈련을 수행할 수 있습니다.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    examples=ratings_transform.outputs['transformed_examples'],
    transform_graph=ratings_transform.outputs['transform_graph'],
    schema=ratings_transform.outputs['post_transform_schema'],
    train_args=tfx.proto.TrainArgs(num_steps=500),
    eval_args=tfx.proto.EvalArgs(num_steps=10),
    custom_config={
        'epochs':5,
        'movies':movies_transform.outputs['transformed_examples'],
        'movie_schema':movies_transform.outputs['post_transform_schema'],
        'ratings':ratings_transform.outputs['transformed_examples'],
        'ratings_schema':ratings_transform.outputs['post_transform_schema']
        })

context.run(trainer, enable_cache=False)

## 모델 내보내기

모델 훈련 후, [Pusher 컴포넌트](https://www.tensorflow.org/tfx/guide/pusher)를 사용하여 모델을 내보낼 수 있습니다.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/tfrs_retrieval')

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

## 예측하기

이제 모델이 있기 때문에 이를 다시 로드하여 예측합니다.

In [ ]:
loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")

## 다음 단계

이 튜토리얼에서, TensorFlow Recommenders 및 TFX로 검색 모델을 구현하는 방법을 배웠습니다. 여기에서 제시된 내용을 확장하려면 [TFX를 사용한 TFRS 순위](https://www.tensorflow.org/recommenders/examples/ranking_tfx) 튜토리얼을 살펴보세요.